In [3]:
import torch

# 1. Charger le fichier texte
with open('exemple.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
text = text.strip().lower()

# 2. Vocabulaire
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

# 3. Encodage
data = torch.tensor([stoi[c] for c in text], dtype=torch.long)
block_size = 64

# 4. Génération de batchs
def get_batch(batch_size=4):
    ix = torch.randint(0, len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x, y

print("Texte encodé. Taille vocabulaire :", vocab_size)

Texte encodé. Taille vocabulaire : 36


In [4]:
import torch.nn as nn
import torch.nn.functional as F

# Définition d’un mini-modèle GPT
class TinyGPT(nn.Module):
    def __init__(self, vocab_size, n_embed=64):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx):
        x = self.embed(idx)
        logits = self.lm_head(x)
        return logits

# Initialisation
model = TinyGPT(vocab_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# Entraînement simple
for step in range(500):
    x, y = get_batch()
    logits = model(x)
    B, T, C = logits.shape
    loss = loss_fn(logits.view(B*T, C), y.view(B*T))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 100 == 0:
        print(f"Étape {step} – Perte : {loss.item():.4f}")

print("✅ Entraînement terminé.")

Étape 0 – Perte : 3.7286
Étape 100 – Perte : 2.4619
Étape 200 – Perte : 2.2383
Étape 300 – Perte : 2.0873
Étape 400 – Perte : 2.0169
✅ Entraînement terminé.


In [5]:
@torch.no_grad()
def generate(model, prompt, max_new_tokens=100):
    model.eval()
    idx = torch.tensor([stoi[c] for c in prompt], dtype=torch.long).unsqueeze(0)  # (1, len)
    
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]  # garder les derniers caractères si trop long
        logits = model(idx_cond)
        logits = logits[:, -1, :]  # (1, vocab_size) → dernier token
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)  # tirage aléatoire
        idx = torch.cat([idx, next_id], dim=1)

    out = ''.join([itos[i.item()] for i in idx[0]])
    return out

# Exemple d'utilisation
prompt = "un matin,"
generated_text = generate(model, prompt, max_new_tokens=200)
print("🔮 Texte généré :\n")
print(generated_text)

🔮 Texte généré :

un matin, mourine. mex, ges maiel?mmellan br.. bsse mmi, dit pprel faisnt lquvin bouivt ninexpantr ll’ait, est, faue saschares, à bo loier à sintéritr. it, : oien ost letéruinex, mppaége lit flatuimitisvel’at 
